In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/LLM-with-RAG/notebook


In [2]:
import sys
sys.path.append("")
import os
import torch
import pickle
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as Soup


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PyPDFLoader,
    RecursiveUrlLoader, 
    DirectoryLoader,
    UnstructuredHTMLLoader,
    UnstructuredWordDocumentLoader,
    BSHTMLLoader,
    UnstructuredImageLoader,
    Docx2txtLoader,
    TextLoader,
    SeleniumURLLoader,
    YoutubeLoader,
    UnstructuredURLLoader,
    NewsURLLoader,
    UnstructuredPowerPointLoader,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.text_splitter import TokenTextSplitter

from src.utils import *

In [3]:
DATA_PATH="web_data"
db_faiss_path = "vectorstores/db_faiss/"
device = take_device()

Selected device: cuda


In [4]:
a = os.path.splitext(os.path.basename('data/sample/2_Software Technical Review.pdf'))[1]
a

'.pdf'

will have csv file that has backup_date, file_path, extension ".ppt", 

In [10]:
class VectorDatabase:
    def __init__(self, 
                    data_path: str = 'data/web_data/', 
                    data_index_path: str = 'data/data_index.csv',
                     db_faiss_path: str = 'data/vectorstores/db_faiss/', 
                     model_config_path: str = 'config/model_config.yaml',
                     data_url_path: str = 'config/data_config.yaml',
                     device = 'cpu'):
        
        self.data_path = data_path
        self.db_faiss_path = db_faiss_path
        self.device = device
        self.data_index_path = data_index_path
        self.data_index = self._read_data_index()
        
        # check_path(self.data_index_path)
        
        self.data_url = config_parser(data_config_path = data_url_path)
        self.model_config = config_parser(data_config_path = model_config_path)
        self.embeddings = HuggingFaceEmbeddings(model_name=self.model_config.get('embedding_model'),
                                model_kwargs={'device': take_device()})
        
    def _read_data_index(self):
        if os.path.exists(self.data_index_path):
            # Load the existing data index
            data_index = pd.read_csv(self.data_index_path)
        else:
            # Create a new DataFrame if the file doesn't exist
            data_index = pd.DataFrame(columns=['file_path', 'extension', 'backup_date'])
        return data_index
    
    def is_file_in_db(self, file_path):
        return file_path in self.data_index['file_path'].values

    @timeit
    def read_file_data(self):
        documents = []
        file_path_list = []
        for f in os.listdir(self.data_path):
            path = f'{self.data_path}/' + f
            try:
                if f.endswith(".pdf") and not self.is_file_in_db(path):
                    loader = PyPDFLoader(path)
                    documents.extend(loader.load())
                    file_path_list.append(path)

                elif f.endswith(".html") and not self.is_file_in_db(path):
                    loader = BSHTMLLoader(path)
                    documents.extend(loader.load())
                    file_path_list.append(path)

                elif f.endswith(".docx") and not self.is_file_in_db(path):
                    loader = Docx2txtLoader(path)
                    documents.extend(loader.load())
                    file_path_list.append(path)

                elif (f.endswith(".txt") or f.endswith(".md")) and not self.is_file_in_db(path):
                    loader = TextLoader(path)
                    documents.extend(loader.load())
                    file_path_list.append(path)
                
                elif f.endswith(".ppt") or f.endswith(".pptx") and not self.is_file_in_db(path):
                    
                    if f.endswith(".ppt"):
                        # Change the file name to "name.pptx"
                        new_path = os.path.join(os.path.dirname(path), os.path.splitext(f)[0] + ".pptx")

                        # Rename the file
                        os.rename(path, new_path)
                    loader = UnstructuredPowerPointLoader(path)
                    documents.extend(loader.load())
                    file_path_list.append(path)

            except Exception as e:
                print("issue with ",f)
                print('Error:',e)
                pass

            single_url_list = self.data_url.get('single_url_list')
            print('single_url_list', single_url_list)
            if single_url_list and not self.is_file_in_db(path):
                loader = NewsURLLoader(urls=single_url_list, 
                                    post_processors=[clean_extra_whitespace],)
                documents.extend(loader.load())
                file_path_list.extend(single_url_list)
                print('file_path_list', file_path_list)


            youtube_url_list = self.data_url.get('youtube_url_list')
            if youtube_url_list and not self.is_file_in_db(path):
                for link in youtube_url_list:
                    loader = YoutubeLoader.from_youtube_url(
                        link, add_video_info=False
                    )
                    documents.extend(loader.load())
                    file_path_list.append(link)
            
            recursive_url_list = self.data_url.get('recursive_url_list')
            if recursive_url_list and not self.is_file_in_db(path):
                for link in recursive_url_list:
                    loader = RecursiveUrlLoader(
                        url=link, max_depth=2,
                        extractor=lambda x: Soup(x, "html.parser").text,
                    )
                    documents.extend(loader.load())
                    file_path_list.append(link)

        return documents, file_path_list

    
    def _split_documents(self, documents):
        text_splitter = TokenTextSplitter(chunk_size=self.model_config.get('splitter_chunk_size'),
                                           chunk_overlap=self.model_config.get('splitter_chunk_overlap'))
        texts = text_splitter.split_documents(documents)
        return texts
    

    def save_vector_database(self, texts):
        new_db = FAISS.from_documents(texts, self.embeddings)
        try:
            old_db = self.load_vector_db()
            old_db.merge_from(new_db)
            old_db.save_local(self.db_faiss_path)
        except:
            new_db.save_local(self.db_faiss_path)

        print(f"Vector database saved in {self.db_faiss_path}.")



    def save_data_index(self, file_path_list):
        data_index = self._read_data_index()
        current_date = datetime.now().strftime('%Y-%m-%d')
        file_info = [(path, os.path.splitext(os.path.basename(path))[1][1:], current_date) for path in file_path_list]
        new_data = pd.DataFrame(file_info, columns=['file_path', 'extension', 'backup_date'])

        # Append the new data to the existing data index
        data_index = pd.concat([data_index, new_data], ignore_index=True)
        data_index.to_csv(self.data_index_path, index=False)
        print (f'Data index saved to CSV in {self.data_index_path}')

    @timeit
    def create_vector_db(self):
        documents, file_path_list = self.read_file_data()
        if documents:
            texts = self._split_documents(documents)
            self.save_vector_database(texts)
            self.save_data_index(file_path_list)
            print(f"Done processing! {len(documents)} documents processed.")
        return documents, file_path_list
    
    @timeit
    def load_vector_db(self):
        try:
            database = FAISS.load_local(self.db_faiss_path, self.embeddings)
        except Exception as e:
            print ('Error:',e)
            return None
        return database
    
    # def read_new_data(self):
    #     documents, file_path_list = self.read_file_data()
    #     texts = self.split_documents(documents)
    #     new_db = FAISS.from_documents(texts, self.embeddings)
    #     return new_db

    # def update_vector_data(self, new_db):
    #     old_db = self.load_vector_db()
    #     old_db.merge_from(new_db)
    #     old_db.save_local(self.db_faiss_path)
    #     print(f"Done processing! {len(new_db)} documents processed.")

    # def add_data(self, file_path:str, loader):
    #     documents = []
    #     file_path_list = []
    #     documents.extend(loader.load())
    #     file_path_list.append(file_path)

    #     texts = self._split_documents(documents)
    #     new_db = FAISS.from_documents(texts, self.embeddings)
    #     old_db = self.load_vector_db()
    #     old_db.merge_from(new_db)
    #     old_db.save_local(self.db_faiss_path)
    #     print(f"Done processing! {len(documents)} documents processed.")

In [11]:
vector_db = VectorDatabase(data_path = 'data/sample/')

Selected device: cuda


In [12]:
documents, file_path_list = vector_db.create_vector_db()

single_url_list ['https://vnexpress.net/viet-nam-len-tieng-ve-viec-trung-quoc-chiem-doat-hoang-sa-4703117.html']
single_url_list ['https://vnexpress.net/viet-nam-len-tieng-ve-viec-trung-quoc-chiem-doat-hoang-sa-4703117.html']
read_file_data took 0.00 seconds to execute.
create_vector_db took 0.00 seconds to execute.


In [8]:
file_path_list

['data/sample//3_White-box testing.pdf',
 'https://vnexpress.net/viet-nam-len-tieng-ve-viec-trung-quoc-chiem-doat-hoang-sa-4703117.html',
 'data/sample//sample.pdf',
 'https://vnexpress.net/viet-nam-len-tieng-ve-viec-trung-quoc-chiem-doat-hoang-sa-4703117.html']

In [9]:
db = vector_db.load_vector_db()

load_vector_db took 0.00 seconds to execute.


In [43]:
db.docstore.__dict__

{'_dict': {'7c73bc2c-0ba6-456b-993b-9755e2ddca20': Document(page_content='1DATA ENGINEERING\nCOURSE OVERVIEW\nTrong Nhan Phan, PhD\n', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 0}),
  'd01ec284-476f-4f56-8ea2-2d540e2c8ae3': Document(page_content='CONTACT\n❑Lecturer: Phan Trọng Nhân, PhD\n❑Faculty: Computer Science and Engineering\n❑Department: Information Systems\n❑Email: nhanpt@hcmut.edu.vn\n❑Course site: LMS\n2', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 1}),
  'e04d45b0-2940-488d-a464-7bfabab9a6b1': Document(page_content='COURSE INTRODUCTION\n❑Subject: Data Engineering\n❑Number: 055240\n❑Credit: 3\n❑Period: 70.5\n❑Prerequisite :None\n❑Class Periods :13weeks\n❑Course Syllabus\n3', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 2}),
  '4d5319aa-d2c4-4ffa-861f-e826db9c9f1f': Document(page_content='COURSE AIMS\n◼Database analysis and design methodologies\nincluding relational data mode

In [30]:
db = vector_db.load_vector_db()
db.docstore._dict

{'cb2f0ddf-8cee-4bb9-8e01-a946b2fd7c13': Document(page_content='1DATA ENGINEERING\nCOURSE OVERVIEW\nTrong Nhan Phan, PhD\n', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 0}),
 '4ce9d22b-b523-4710-a227-6176bddda3b1': Document(page_content='CONTACT\n❑Lecturer: Phan Trọng Nhân, PhD\n❑Faculty: Computer Science and Engineering\n❑Department: Information Systems\n❑Email: nhanpt@hcmut.edu.vn\n❑Course site: LMS\n2', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 1}),
 '1f9769ef-35db-47b3-bf89-ec016750f319': Document(page_content='COURSE INTRODUCTION\n❑Subject: Data Engineering\n❑Number: 055240\n❑Credit: 3\n❑Period: 70.5\n❑Prerequisite :None\n❑Class Periods :13weeks\n❑Course Syllabus\n3', metadata={'source': 'data/sample//Chapter 0-Course introduction.pdf', 'page': 2}),
 '33e6dbbe-cd79-4bad-bb57-4832c9f8f99e': Document(page_content='COURSE AIMS\n◼Database analysis and design methodologies\nincluding relational data model and NoSQL\n

In [31]:
def create_vector_db(DATA_PATH: str = 'web_data/', 
                     DB_FAISS_PATH: str = 'vectorstores/db_faiss/', 
                     device = 'cpu'):
    documents = []
    data = config_parser(data_config_path = 'config/data_config.yaml')

    for f in os.listdir(DATA_PATH):
        path = f'{DATA_PATH}/' + f
        try:
            if f.endswith(".pdf"):
                loader = PyPDFLoader(path)
                documents.extend(loader.load())

            elif f.endswith(".html"):
                loader = BSHTMLLoader(path)
                documents.extend(loader.load())

            elif f.endswith(".docx"):
                loader = Docx2txtLoader(path)
                documents.extend(loader.load())

            elif f.endswith(".txt") or f.endswith(".md"):
                loader = TextLoader(path)
                documents.extend(loader.load())
            
            elif f.endswith(".ppt") or f.endswith(".pptx"):
                
                if f.endswith(".ppt"):
                    # Change the file name to "name.pptx"
                    new_path = os.path.join(os.path.dirname(path), os.path.splitext(f)[0] + ".pptx")

                    # Rename the file
                    os.rename(path, new_path)
                loader = UnstructuredPowerPointLoader(path)
                documents.extend(loader.load())


            single_url_list = data.get('single_url_list')
            loader = NewsURLLoader(urls=single_url_list, 
                                post_processors=[clean_extra_whitespace],)
            documents.extend(loader.load())

            youtube_url_list = data.get('youtube_url_list')
            for link in youtube_url_list:
                loader = YoutubeLoader.from_youtube_url(
                    link, add_video_info=False
                )
                documents.extend(loader.load())

            
            recursive_url_list = data.get('recursive_url_list')
            for link in recursive_url_list:
                loader = RecursiveUrlLoader(
                    url=link, max_depth=2,
                    extractor=lambda x: Soup(x, "html.parser").text,
                )
                documents.extend(loader.load())

        except Exception as e:
            print("issue with ",f)
            print('Error:',e)
            pass


    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_splitter = TokenTextSplitter(chunk_size=256,
                                           chunk_overlap=10)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': device})
    
    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)
    
    print(f"Done processing! {len(documents)} documents processed.") 

In [32]:
db = create_vector_db(self.DATA_PATH, db_faiss_path, device)


NameError: name 'self' is not defined

In [ ]:
db.docstore._dict

{'96d42b41-e390-48af-8f0f-8010e5486d76': Document(page_content='CH3. WHITE-BOX TESTINGSOFTWARE TESTINGCO3015 / CO5252', metadata={'source': 'web_data/3_White-box testing.pdf', 'page': 0}),
 'de8df8ce-e6fa-4809-beb6-d1002229ea21': Document(page_content='Content■Control flow testing–Execution path–Control flow graph–Control flow testing■Data flow testing–Data flow testing–Variable state transition–Data flow graphJan-2024Software Testing -White-box testing2', metadata={'source': 'web_data/3_White-box testing.pdf', 'page': 1}),
 '5071b795-16bd-4c49-8f4a-7991934923c9': Document(page_content='White-box tes.ng■White Box Testing is software testing technique in which internal structure, designand codingof software are tested to verify flow of input-output and to improve design, usability and security. ■In white box testing, code is visible to testersso it is also called Clear box testing, Open box testing, Transparent box testing, Code-based testing and Glass box testing.■“The foundation of wh

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device':device})
db2 = FAISS.load_local(db_faiss_path, embeddings,index_name='index')

In [ ]:
db2.docstore._dict

{'96d42b41-e390-48af-8f0f-8010e5486d76': Document(page_content='CH3. WHITE-BOX TESTINGSOFTWARE TESTINGCO3015 / CO5252', metadata={'source': 'web_data/3_White-box testing.pdf', 'page': 0}),
 'de8df8ce-e6fa-4809-beb6-d1002229ea21': Document(page_content='Content■Control flow testing–Execution path–Control flow graph–Control flow testing■Data flow testing–Data flow testing–Variable state transition–Data flow graphJan-2024Software Testing -White-box testing2', metadata={'source': 'web_data/3_White-box testing.pdf', 'page': 1}),
 '5071b795-16bd-4c49-8f4a-7991934923c9': Document(page_content='White-box tes.ng■White Box Testing is software testing technique in which internal structure, designand codingof software are tested to verify flow of input-output and to improve design, usability and security. ■In white box testing, code is visible to testersso it is also called Clear box testing, Open box testing, Transparent box testing, Code-based testing and Glass box testing.■“The foundation of wh